<a href="https://colab.research.google.com/github/rohitkhadka1/AI-ML-DS/blob/main/LLM_App_for_Creative_Mathematics_Problem_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import asyncio
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [ ]:
# Load Mistral-7B (quantized to 4-bit)
model_1_name = "mistralai/Mistral-7B-v0.1"
tokenizer_1 = AutoTokenizer.from_pretrained(model_1_name)

model_1 = AutoModelForCausalLM.from_pretrained(
    model_1_name,
    torch_dtype=torch.float16,  # Use FP16 for lower VRAM usage
    load_in_4bit=True,  # Quantization for memory efficiency
    device_map="auto"   # Automatically place model parts across CPU/GPU
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Load a second model (Example: LLaMA-2-7B)
model_2_name = "meta-llama/Llama-2-7b-hf"
tokenizer_2 = AutoTokenizer.from_pretrained(model_2_name)

model_2 = AutoModelForCausalLM.from_pretrained(
    model_2_name,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
summarizer_model = model_1

In [ ]:
prompt_template = PromptTemplate(
    template="Generate a diverse set of mathematics problems based on the following user input: {question}. Ensure that:\n"
             "- Each problem explores a **distinct mathematical concept** or application.\n"
             "- The set includes problems from **various domains** (e.g., algebra, calculus, probability, geometry, combinatorics, number theory, etc.).\n"
             "- Problems vary in **difficulty levels**, from introductory to advanced.\n"
             "- The wording is **unique and engaging**, avoiding redundancy.\n"
             "- If possible, include **real-world context** to enhance intuition.\n\n"
             "**Example:**\n"
             "User Input: 'Generate an interesting probability problem.'\n"
             "Output:\n"
             "1. *The Birthday Paradox*: In a room of 23 people, what is the probability that at least two of them share the same birthday?\n"
             "2. *Monty Hall Problem*: You are on a game show with three doors, behind one of which is a car and behind the others are goats. After you pick a door, the host (who knows what’s behind each door) opens another door revealing a goat. Should you switch your choice?\n"
             "3. *Flipping Coins*: A fair coin is flipped 10 times. What is the probability of getting exactly 5 heads?\n\n"
             "Now, generate a set of problems based on the following query:\n"
             "User Input: {question}\nGenerated Problems:",
    input_variables=["question"]
)

summary_template = PromptTemplate(
    template="Given the following set of generated math problems, improve them by:\n"
             "- Removing redundant or repetitive phrasing.\n"
             "- Making each problem clearer and more engaging.\n"
             "- Ensuring logical progression in difficulty.\n"
             "- Formatting them properly for better readability.\n"
             "\nGenerated Problems: {responses}\nRefined Problems:",
    input_variables=["responses"]
)


In [ ]:
# Create HuggingFacePipeline instances for both models
llm_1 = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model_1,
    tokenizer=tokenizer_1,
    torch_dtype=torch.float16,
    device_map="auto",
))

Device set to use cuda:0
<ipython-input-9-6271e4592eab>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_1 = HuggingFacePipeline(pipeline=pipeline(


In [ ]:
llm_2 = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model_2,
    tokenizer=tokenizer_2,
    torch_dtype=torch.float16,
    device_map="auto",
))

Device set to use cuda:0


In [ ]:
# Define LLM Chains
chain_1 = LLMChain(llm = llm_1, prompt = prompt_template)
chain_2 = LLMChain(llm = llm_2, prompt = prompt_template)
summary_chain = LLMChain(llm = llm_1, prompt = summary_template)


<ipython-input-11-4bca17f2fc86>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_1 = LLMChain(llm = llm_1, prompt = prompt_template)


In [ ]:
# Async Execution
async def run_models(question):
    response_1_task = asyncio.create_task(chain_1.arun(question=question))
    response_2_task = asyncio.create_task(chain_2.arun(question=question))

    response_1, response_2 = await asyncio.gather(response_1_task, response_2_task)
    summary = await summary_chain.arun(responses=f"Model 1: {response_1}\nModel 2: {response_2}")

    return {"model_1": response_1, "model_2": response_2, "summary": summary}

In [ ]:
# Example Usage
user_query = "Create engaging probability problems that connect to real-world scenarios."

responses = await run_models(user_query)

/usr/lib/python3.11/asyncio/events.py:84: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  self._context.run(self._callback, *self._args)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# Print Outputs
print("\nModel 1 Response:\n", responses["model_1"])


Model 1 Response:
 Generate a diverse set of mathematics problems based on the following user input: Create engaging probability problems that connect to real-world scenarios.. Ensure that:
- Each problem explores a **distinct mathematical concept** or application.
- The set includes problems from **various domains** (e.g., algebra, calculus, probability, geometry, combinatorics, number theory, etc.).
- Problems vary in **difficulty levels**, from introductory to advanced.
- The wording is **unique and engaging**, avoiding redundancy.
- If possible, include **real-world context** to enhance intuition.

**Example:**
User Input: 'Generate an interesting probability problem.'
Output:
1. *The Birthday Paradox*: In a room of 23 people, what is the probability that at least two of them share the same birthday?
2. *Monty Hall Problem*: You are on a game show with three doors, behind one of which is a car and behind the others are goats. After you pick a door, the host (who knows what’s behin

In [ ]:
print("\nModel 2 Response:\n", responses["model_2"])


Model 2 Response:
 Generate a diverse set of mathematics problems based on the following user input: Create engaging probability problems that connect to real-world scenarios.. Ensure that:
- Each problem explores a **distinct mathematical concept** or application.
- The set includes problems from **various domains** (e.g., algebra, calculus, probability, geometry, combinatorics, number theory, etc.).
- Problems vary in **difficulty levels**, from introductory to advanced.
- The wording is **unique and engaging**, avoiding redundancy.
- If possible, include **real-world context** to enhance intuition.

**Example:**
User Input: 'Generate an interesting probability problem.'
Output:
1. *The Birthday Paradox*: In a room of 23 people, what is the probability that at least two of them share the same birthday?
2. *Monty Hall Problem*: You are on a game show with three doors, behind one of which is a car and behind the others are goats. After you pick a door, the host (who knows what’s behin

In [ ]:
print("\nFinal Summary:\n", responses["summary"])


Final Summary:
 Given the following set of generated math problems, improve them by:
- Removing redundant or repetitive phrasing.
- Making each problem clearer and more engaging.
- Ensuring logical progression in difficulty.
- Formatting them properly for better readability.

Generated Problems: Model 1: Generate a diverse set of mathematics problems based on the following user input: Create engaging probability problems that connect to real-world scenarios.. Ensure that:
- Each problem explores a **distinct mathematical concept** or application.
- The set includes problems from **various domains** (e.g., algebra, calculus, probability, geometry, combinatorics, number theory, etc.).
- Problems vary in **difficulty levels**, from introductory to advanced.
- The wording is **unique and engaging**, avoiding redundancy.
- If possible, include **real-world context** to enhance intuition.

**Example:**
User Input: 'Generate an interesting probability problem.'
Output:
1. *The Birthday Parad